In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("C:/Users/mekin/OneDrive/Desktop/java/kaggle"):
    for filename in filenames:
        print(os.path.join(dirname, filename))



C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive\Resume.csv
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive\training_data.csv
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\10554236.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\10674770.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\11163645.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\11759079.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\12065211.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\12202337.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\12338274.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\12442909.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\data\data\ACCOUNTANT\12780508.pdf
C:/Users/mekin/OneDrive/Desktop/java/kaggle\archive (1)\d

In [2]:
!pip install transformers

In [3]:
!pip install nltk

In [3]:
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia


^C


In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch built with CUDA: {torch.version.cuda if hasattr(torch.version, 'cuda') else 'None'}")

PyTorch version: 2.5.1
CUDA available: True
PyTorch built with CUDA: 11.8


In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
print(f"CUDA available: {torch.cuda.is_available()}")


CUDA available: True


In [4]:
import pandas as pd
from pypdf import PdfReader
import nltk
from nltk import pos_tag, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import re
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"



nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = "".join(page.extract_text() for page in reader.pages)
    return text

def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    sentences = sent_tokenize(text)
    features = {'feature': ""}
    stop_words = set(stopwords.words("english"))
    for sent in sentences:
        if any(criteria in sent for criteria in ['skills', 'education']):
            words = word_tokenize(sent)
            words = [word for word in words if word not in stop_words]
            tagged_words = pos_tag(words)
            filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
            features['feature'] += " ".join(filtered_words)
    return features

def process_resume_data(df):
    id = df['ID']
    category = df['Category']
    text = extract_text_from_pdf(f"C:/Users/mekin/OneDrive/Desktop/java/kaggle/archive (1)/data/data/{category}/{id}.pdf")
    features = preprocess_text(text)
    df['Feature'] = features['feature']
    return df

def get_embeddings(text, model_name, device, model, tokenizer):
    """Generate embeddings for a text using the provided model and tokenizer"""
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().to("cpu").numpy()
    return embeddings

def print_top_matching_resumes(result_group):
    for i in range(15):
        print("\nJob ID:", i)
        print("Cosine Similarity | Domain Resume | Domain Description")
        print(result_group.get_group(i)[['similarity', 'domainResume', 'domainDesc']])

def main():

    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"CUDA device count: {torch.cuda.device_count()}")
    if torch.cuda.is_available():
        print(f"Current CUDA device: {torch.cuda.current_device()}")
        print(f"CUDA device name: {torch.cuda.get_device_name()}")



    resume_data = pd.read_csv("C:/Users/mekin/OneDrive/Desktop/java/kaggle/archive (1)/Resume/Resume.csv")
    resume_data = resume_data.drop(["Resume_html"], axis=1)
    resume_data = resume_data.apply(process_resume_data, axis=1)
    resume_data = resume_data.drop(columns=['Resume_str'])
    resume_data.to_csv("C:/Users/mekin/OneDrive/Desktop/java/kaggle/archive (1)/processed_resumes.csv", index=False)

    job_description = pd.read_csv("C:/Users/mekin/OneDrive/Desktop/java/kaggle/archive/training_data.csv")
    job_description = job_description[["job_description", "position_title"]][:15]
    job_description['Features'] = job_description['job_description'].apply(lambda x : preprocess_text(x)['feature'])

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    model_name = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.to(device)

    job_desc_embeddings = np.array([get_embeddings(desc, model_name, device, model, tokenizer) 
                                  for desc in job_description['Features']]).squeeze()
    resume_embeddings = np.array([get_embeddings(text, model_name, device, model, tokenizer) 
                                for text in resume_data['Feature']]).squeeze()

    result_df = pd.DataFrame(columns=['jobId', 'resumeId', 'similarity', 'domainResume', 'domainDesc'])

    for i, job_desc_emb in enumerate(job_desc_embeddings):
        similarities = cosine_similarity([job_desc_emb], resume_embeddings)
        top_k_indices = np.argsort(similarities[0])[::-1][:5]
        for j in top_k_indices:
            result_df.loc[i+j] = [i, resume_data['ID'].iloc[j], similarities[0][j], resume_data['Category'].iloc[j], job_description['position_title'].iloc[i]]

    result_df = result_df.sort_values(by='similarity', ascending=False)
    result_group = result_df.groupby("jobId")
    print_top_matching_resumes(result_group)

    # Save the model and tokenizer
    save_path = "saved_bert_model"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    print(f"Model and tokenizer saved to {save_path}")


if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mekin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\mekin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


CUDA available: True
CUDA device count: 1
Current CUDA device: 0
CUDA device name: NVIDIA GeForce RTX 2060 SUPER
Using device: cuda

Job ID: 0
Cosine Similarity | Domain Resume | Domain Description
      similarity            domainResume        domainDesc
629     0.938355    BUSINESS-DEVELOPMENT  Sales Specialist
299     0.930880  INFORMATION-TECHNOLOGY  Sales Specialist
577     0.930776    BUSINESS-DEVELOPMENT  Sales Specialist
1235    0.930246           DIGITAL-MEDIA  Sales Specialist
1045    0.929638                   SALES  Sales Specialist

Job ID: 1
Cosine Similarity | Domain Resume | Domain Description
      similarity   domainResume                  domainDesc
2311    0.908976           ARTS  Apple Solutions Consultant
1048    0.903436          SALES  Apple Solutions Consultant
2151    0.901145        BANKING  Apple Solutions Consultant
1300    0.899512  DIGITAL-MEDIA  Apple Solutions Consultant
1227    0.893605  DIGITAL-MEDIA  Apple Solutions Consultant

Job ID: 2
Cosine Simi

In [5]:
import torch
from transformers import AutoModel, AutoTokenizer

# Set the device (GPU if available, otherwise CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Path to the saved model and tokenizer
save_path = "saved_bert_model"

# Load the saved model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_path)
model = AutoModel.from_pretrained(save_path)
model.to(device)

# Example test text
test_text = "This is a sample sentence to test the saved model."

# Tokenize the input text
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True).to(device)

# Run inference (disable gradients for faster inference)
with torch.no_grad():
    outputs = model(**inputs)
    # For instance, you can use mean pooling over token embeddings:
    embeddings = outputs.last_hidden_state.mean(dim=1)
    
print("Embeddings:", embeddings)


Embeddings: tensor([[-8.4978e-02, -4.7097e-01, -7.6098e-02, -5.3171e-02, -7.9071e-02,
         -1.9292e-01,  4.0640e-01,  1.7338e-01,  4.3455e-02,  1.4082e-01,
         -9.0081e-02, -2.1509e-01, -1.2572e-01,  2.1034e-01, -1.7857e-01,
         -1.5375e-02,  4.3843e-02,  4.3501e-03,  3.9147e-01, -2.8389e-01,
          1.7562e-01,  3.4733e-01, -2.3865e-01,  1.1252e-01,  6.7988e-01,
         -3.5976e-01,  1.1459e-01, -2.1104e-02, -6.0715e-01, -2.5593e-01,
         -8.5816e-02,  4.1879e-01, -2.9019e-01, -3.4820e-01, -2.1062e-01,
         -6.8508e-02,  4.1800e-01, -1.8526e-01,  5.6763e-02,  1.6056e-01,
         -4.8194e-01, -3.6114e-01,  2.0053e-01,  1.7764e-01, -2.3957e-01,
         -3.5941e-01, -3.3824e-02, -4.4391e-01,  9.2817e-02, -2.4100e-01,
         -8.9048e-01,  6.2165e-01,  1.1758e-01,  3.1633e-01, -1.2943e-01,
          5.6998e-01,  2.0229e-01, -7.8106e-01,  2.4636e-02, -3.2846e-01,
          4.5726e-01,  1.5459e-01,  1.1131e-01, -4.2484e-01,  3.2460e-01,
          7.4427e-02,  1.0

In [20]:
job_description_text = """


Job Summary
TechNova Solutions is looking for a skilled Java Developer to design, develop, and maintain scalable applications. The ideal candidate will have experience with Java, Spring Boot, RESTful APIs, and cloud technologies. This role requires strong problem-solving skills and the ability to work in an agile environment.

Responsibilities
Develop and maintain Java-based applications using Spring Boot
Design RESTful APIs and integrate third-party services
Optimize code for performance, scalability, and maintainability
Collaborate with cross-functional teams in an agile environment
Write unit tests and ensure code quality using CI/CD pipelines
Deploy applications on cloud platforms such as AWS or Azure
Requirements
Bachelor’s degree in Computer Science or a related field
3+ years of Java development experience
Proficiency in Spring Boot, Hibernate, and RESTful APIs
Experience with cloud services (AWS, Azure, or GCP)
Familiarity with CI/CD tools like Jenkins or GitHub Actions
Strong problem-solving and analytical skills
Excellent communication and teamwork abilities


"""

resume_text = """

Sales Executive with 8 years of experience in direct sales, lead generation, and revenue optimization. Strong expertise in B2B sales, CRM software, and data-driven sales strategies. Specialized in pipeline management, pricing strategies, and contract negotiations with a proven track record of exceeding sales targets in the technology and finance sectors.


CRM Systems: Salesforce, HubSpot, Zoho CRM
Sales Forecasting: Market trend analysis, revenue prediction models
Lead Generation: Cold outreach automation, inbound sales funnel optimization
Pricing & Negotiation: Competitive pricing analysis, deal structuring
B2B Sales Strategies: Enterprise account management, SaaS sales pipeline
Data Analytics: Sales performance metrics, A/B testing for conversion rates
Contract Management: Client acquisition, legal agreement structuring
"""


In [21]:

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity


# Set up device and load the saved model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
save_path = "C:/Users/mekin/OneDrive/Desktop/java/resume-match-bert"
tokenizer = AutoTokenizer.from_pretrained(save_path)
model = AutoModel.from_pretrained(save_path)
model.to(device)

def get_embedding(text):
    """Get mean-pooled embedding for the input text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # Mean pooling over the token embeddings
    embedding = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()
    return embedding

# Example texts

# Generate embeddings
resume_embedding = get_embedding(resume_text)
job_embedding = get_embedding(job_description_text)

# Compute cosine similarity
similarity = cosine_similarity(resume_embedding, job_embedding)[0][0]
print("Cosine Similarity Score:", similarity)

# Optionally, you can convert the similarity to a percentage
match_percentage = similarity * 100
print(f"Resume-Job Match: {match_percentage:.2f}%")


Cosine Similarity Score: 0.8587339
Resume-Job Match: 85.87%
